In [ ]:
!pip install python-telegram-bot==13.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.3
    Uninstalling cachetools-5.3.3:
      Successfully uninstalled cachetools-5.3.3


In [ ]:
!pip install llama_cpp_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.4 MB/s eta 0:00:00
  Created wheel for llama_cpp_python: filename=llama_cpp_python-0.2.75-cp310-cp310-linux_x86_64.whl size=3688356 sha256=cc0580eeb39e8b84a10505dce560ca8e030d6b3341b9cc46d2d2e7b9e56fc11c
  Stored in directory: /root/.cache/pip/wheels/5e/df/9a/e4bb2e48bfa64fb174f0f786296c8507dbebea2a112f1adf8d
Successfully built llama_cpp_python


In [ ]:
! pip install pytelegrambotapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
from llama_cpp import Llama
from llama_cpp.llama_tokenizer import LlamaHFTokenizer

In [ ]:
import telebot

In [ ]:
API_TOKEN = '7104183144:AAHz81RRtZg1RSfm-02x8ARv3eZuJhhbPcs'
bot = telebot.TeleBot(API_TOKEN)

In [ ]:
import sqlite3

# Conectar ao banco de dados SQLite
conn = sqlite3.connect('milkshakes.db')
c = conn.cursor()

# Criar tabela de sabores de milkshakes
c.execute('''CREATE TABLE IF NOT EXISTS milkshake_flavors
             (id INTEGER PRIMARY KEY, flavor TEXT)''')

# Inserir alguns tipos de milkshake na tabela de sabores
milkshake_flavors = [
    ('Chocolate',),
    ('Morango',),
    ('Baunilha',),
    ('Caramelo',),
    ('Ovomaltine',),
    ('KinderOvo',),
    ('Banana',)
]

c.executemany('INSERT INTO milkshake_flavors (flavor) VALUES (?)', milkshake_flavors)
# Salvar as alterações e fechar a conexão
conn.commit()
conn.close()


In [ ]:
import sqlite3

# Função para conectar ao banco de dados
def connect_db():
    return sqlite3.connect('milkshakes.db')

# Função para buscar os tipos de milkshake disponíveis
def get_milkshake_types():
    conn = connect_db()
    c = conn.cursor()
    c.execute('SELECT flavor FROM milkshake_flavors')
    flavors = c.fetchall()
    conn.close()
    return [flavor[0] for flavor in flavors]

# Função para buscar o preço de um milkshake específico
def get_milkshake_price(flavor):
    conn = connect_db()
    c = conn.cursor()
    c.execute('SELECT price FROM milkshake_prices JOIN milkshake_flavors ON milkshake_prices.flavor_id = milkshake_flavors.id WHERE flavor = ?', (flavor,))
    price = c.fetchone()
    conn.close()
    print("Price for", flavor, ":", price)  # Instrução de depuração
    if price is not None:
        return price[0]
    else:
        return None



# Função para adicionar um milkshake à lista
def add_to_list(milkshake, milkshake_list, price_list):
    price = get_milkshake_price(milkshake)
    if price is not None:
        milkshake_list.append(milkshake)
        price_list.append(price)
    else:
        print(f"Preço não encontrado para o milkshake: {milkshake}")

# Função para calcular o custo total dos milkshakes na lista
def calculate_total_cost(price_list):
    total_cost = sum(price_list)
    return total_cost

chosen_flavors = []  # Lista para armazenar os sabores escolhidos
chosen_prices = []  # Lista para armazenar os preços dos sabores escolhidos

def process_message(message):
    milkshake_types = get_milkshake_types()

    # Verifica se o usuário está solicitando sabores disponíveis
    if "sabores" in message.text.lower():
        response = f"Olá! Temos os seguintes sabores de milkshake disponíveis: {', '.join(milkshake_types)}. Por favor, liste os sabores desejados."
        return response

    # Verifica se o usuário está adicionando sabores à lista
    elif message.text.lower() in [flavor.lower() for flavor in milkshake_types]:
        chosen_flavor = message.text.lower()
        chosen_flavors.append(chosen_flavor)
        response = f"Você adicionou '{chosen_flavor}' à sua lista de pedidos. Se desejar mais algum sabor, por favor, liste-o. Caso contrário, diga 'confirmar'."
        return response

    # Verifica se o usuário deseja confirmar a lista de pedidos
    elif "confirmar" in message.text.lower():
        if chosen_flavors:
            response = f"Você confirmou os seguintes sabores: {', '.join(chosen_flavors)}. Um atendente virá em breve para tratar dos pagamentos. Obrigado por escolher nossos milkshakes!"
            return response
        else:
            response = "Você ainda não selecionou nenhum sabor. Por favor, liste os sabores desejados."
            return response

    else:
        response = "Desculpe, não entendi. Por favor, liste os sabores desejados ou confirme sua lista com a palavra 'confirmar'."
        return response



# Integrar com o manipulador de mensagens do bot
@bot.message_handler(func=lambda message: True)
def reply_to_message(message):
    response = process_message(message)
    if response:
        bot.reply_to(message, response)

In [ ]:
bot.polling()